# ANALYSE LOCATION DATA IN SNOWFLAKE
This notebook will take you through how you can use location data to perform spatial calculations, joins, and visualise the data using the popular **Pydeck** python package.  We will be using the freely available data on the market place provided by **Ordnance Survey** and buildings data provided by **Carto Overture Maps**. 

## Datasets needed to complete this notebook

#### All datasets are available on the Snowflake Marketplace

- Ordnance Survey - Urban Extents for Cities, Towns and Vilages
- Ordnance Survey - Postcodes, Place Names and Road Names
- Ordnance Survey - Road Network Great Britain - Open Roads
- Carto - Overture Maps Buildings

Please run the next cell to begin

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pydeck as pdk
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


### 1. GEOJSON DATA FORMATS
Snowflake can store and interopolate between the following data formats: 

-   **GeoJSON**:
```json
{   "coordinates": [     -7.390351649999999e+01,     4.074499730000000e+01   ],   "type": "Point" }

```
-   **WKT (Well Known Text)**
```text
POINT(52.3418 2.2776)
```
-   **WKB (Well Known Binary)**

```binary
1,1,0,0,0,235,226,54,26,192,43,74,64,241,244,74,89,134,56,2,64
```

You can decide how you would like the data to be stored and can interchange between them using the following functions: 

- **ST_WKT**, 
- **ST_ASWKB**
- **ST_ASGEOJSON**.  

You can also store this as planar (Euclidean, Cartesian) coordinate system using an SRID of your choice.

Run the **SQL Cell** below to see how these functions work in action:

In [ ]:
SELECT GEOGRAPHY, ST_ASWKT(GEOGRAPHY), ST_ASWKB(GEOGRAPHY), ST_ASGEOJSON(GEOGRAPHY) FROM POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL LIMIT 1

##### GEOGRAPHY AND GEOMETRY

- **GEOGRAPHY** - The Geography column models the earth as though it were a perfect sphere and follows the WGS 84 standard.

- **GEOMETRY** - The GEOMETRY data type represents features in a planar (Euclidean, Cartesian) coordinate system. The units of the X and Y are determined by the spatial reference system (SRS) associated with the GEOMETRY object. The spatial reference system is identified by the spatial reference system identifier (SRID) number.  Snowflake supports the transformation between systems using the **ST_TRANSFORM**  function.

Run the **Python Cell** below to see the difference between GEOGRAPHY AND GEOMETERY.  You will also find out how 'ST_TRANSFORM' works.   The python cell intrduces **Snowpark Data Frames**.  All SQL functions (both user defined and native) are available in both SQL and Python environments.  This Example is using how you can convert from geography to geometry as well as converting between two grid systems.

In [ ]:
names = session.table('POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL')
names.select('NAME1','GEOGRAPHY')
names.limit(5).select('NAME1','GEOGRAPHY',call_function('ST_TRANSFORM',to_geometry('GEOGRAPHY'),4326,27700).alias('GEOMETRY'))

As previously stated before, you can interchange between python and sql without losing functionality.

Run the cell below to view the same results, this time in SQL.

In [ ]:
SELECT NAME1,GEOGRAPHY, ST_TRANSFORM(TO_GEOMETRY(GEOGRAPHY),4326,27700) GEOMETRY FROM (
select * from POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL limit 5)

And the geography can be rendered in formats such as WKT and WKB as well.  WKT is an easy format to read - look at the difference between the WKT for the geography column vs the Geometry Column. 

In [ ]:
select 
TO_GEOGRAPHY(GEOGRAPHY) GEO,
ST_ASWKT(GEO) WKT_GEOGRAPHY,
ST_ASWKT(TO_GEOMETRY(GEOMETRY)) WKT_GEOMETRY_BNG


FROM {{transform_sql}}

# 2 - Points line Strings and Polygons
Snowflake Supports Points, Linestrings and Polygons for both **Geography** and **Geometry** Data Types.  Let's have a look at these in isolation

## 2a. Looking at Points

The dataset you have been looking at briefly consist of millions of points - each reference a place somewhere in the United Kingdom. The Geography format would always contain a **Latitude** for each X and **Longitude** for each Y coordinate inside every point.  The Geometry format using the British National Grid would also contain an **Eastings** for each X and **Northings** for each Y coordinate inside every point.

It is easy to extract the coordinates from the point by using the functions **ST_X** and **ST_Y**.  Visualisation tools such as **st.map** and **pydeck** require the Latitude and Longitude rather than the point itself - however, you may want to retain the Geography column for other geospatial calculations.  Snowflake supports many geospatial calculations.


Below, you are instantantly looking at a sample of points from the Ordnance Survey Names table.  The simplist map to render is using the built in streamlit module (**st.map**).  This, however as limitations.  For the rest of the lab you will be leveraging **pydeck**.  You will note that the data has been filtered based on the 'type' of point. 

Below filters the dataset using a streamlit select box (**st.selectbox**).  All of the visualisations you will see in this lab will be referencing the Latitude and Longitude coordinate system (**WGS 84 or  SRID 4326**)

In [ ]:
names = session.table('POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL')
types = names.select('TYPE').distinct()
selectedtypes = st.selectbox('Select Type',types)
names = names.with_column('LAT',call_function('ST_Y',col('GEOGRAPHY')))
names = names.with_column('LON',call_function('ST_X',col('GEOGRAPHY')))
namesf = names.filter(col('TYPE')==selectedtypes)

r_names = namesf.sample(0.1).limit(1000)

st.map(r_names.to_pandas())
st.markdown('Here is the data coming from the snowflake dataframe.  the  **GEOGRAPHY** field can be used for a variety of geo based calculations')
st.dataframe(r_names.select('NAME1','GEOGRAPHY','LAT','LON'))

If you need to create a point from Latitude and Longitude, use the **ST_MAKEPOINT** to combine LAT and LON to a geography point, or **ST_MAKEGEOPOINT** to convert X and Y to a geometry point

In [ ]:
r_names.select(call_function('ST_MAKEPOINT',col('LON'),col('LAT')).alias('POINT')).limit(1)

Lets select the fields within the dataset that we want to use.  We are referencing the previously created snowpark dataframe.

In [ ]:
snames = names.select('NAME1',
                      'NAME2',
                      'NAME2_LANG',
                      'TYPE',
                      'LOCAL_TYPE',
                      'POSTCODE_DISTRICT',
                      'POPULATED_PLACE',
                      'DISTRICT_BOROUGH',
                      'COUNTY_UNITARY',
                      'REGION',
                      'LAT',
                      'LON',
                      'GEOGRAPHY')
snames.limit(10)


##### Using Pydeck to visualise the results
Here, I have decided to use pydeck to visualise the results becase it has a lot more flexibility than st.map such as: 
- creating tooltips, 
- rendering multi layer maps
- rendering points, linestrings, polygons and H3 cells


For this first map I will take you though each part of the code step by step

#### Step 1 - creating the map dataset and the centerpoint
All the data that pydeck needs is in a pandas dataframe.  Any Snowpark Dataframe can be converted to Pandas with the **to_pandas()** funtion. You will see that the dataframe contains a filter and this filter is determined by a streamlit selectbox.

You also need to tell the map where the focal point is.  This is normally the **center** of all the locations.  There are many techniques to work out the center - for points, a nice simple way is by finding the average latitude and average longitude for all the points.  For polygons you can use **ST_CENTROID** to work out the center of the polygon.  You could also use **ST_ENVELOPE** to create a bounding box - then work out the center of  that bounding box.  This is useful if you have a variety of geographic shapes and want to work out the overall center.  Solving this will make your map dynamic as you filter.

In [ ]:
types = names.select('TYPE').distinct()
selectedtypes = st.selectbox('Select Type',types,1)
snamesf = snames.filter(col('TYPE')==selectedtypes)
center = snames.agg(avg('LAT'),avg('LON'))
LAT = center.collect()[0][0]
LON = center.collect()[0][1]
st.write(center)
placespd = snamesf.sample(0.5).limit(15000).to_pandas()

#### Step 2 - Create a Layer

The layer properties determine how to treat the layer.  It may be a polygon, point,path or H3 (there are also other options).  Each layer type will expect properties.  the properties can either be 'hard coded' or generated from the dataset.  In this case, LON and LAT is in the dataset so pydeck will replace the word LON and LAT with the actual values from the dataframe.  the radius however is hardcoded (but it doesn't have to be)

In [ ]:
landcover_l = pdk.Layer(
            'ScatterplotLayer',
            data=placespd,
            get_position='[LON, LAT]',
            get_color='[41,181,232]',
            get_radius=600,
            pickable=True)


### Step 3 - Render the map.

Here, you will create a pydeck visualisation, which is wrapped around a **streamlit pydeck** plugin object.  this is where the center LAT and LON is situated, as well as what layers you want to include.  We only have one layer this time which is land cover.

Finally every pydeck visual has the option to have a tooltip.  The tooltip will only display if **pickable** in the layer is set to **True**. You can parameterise the tooltip using {YOUR_FIELD}. Wherever pydeck sees a matching field in your dataset, it will replace the parameter with the value.

In [ ]:
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=400
        ),
    
layers= [landcover_l], tooltip = {'text':"Place Name: {NAME1}, Type: {TYPE}"}

))

Simply switching the scatterplot layer type to a heatmap type will give an alternative effect

In [ ]:
### dropdown list creation
types = names.select('TYPE').distinct()
selectedtypes = st.selectbox('Select Type',types,2)

## dataframe selection
snamesf = snames.filter(col('TYPE')==selectedtypes)

## center point generation
center = snames.agg(avg('LAT'),avg('LON'))
LAT = center.collect()[0][0]
LON = center.collect()[0][1]
st.write(center)

## convert dataframe to pandas dataframe
placespd = snamesf.sample(0.5).limit(15000).to_pandas()


## create a heatmap layer
landcover_l = pdk.Layer(
            'HeatmapLayer',
            data=placespd,
            get_position='[LON, LAT]',
            get_color='[41,181,232]',
            get_radius=600,
            pickable=True)

#### render the map to include the layer
    
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=4,
        height=400
        ),
    
layers= [landcover_l], tooltip = {'text':"Place Name: {NAME1}, Type: {TYPE}"}

))

#### Points on a Multi layer Map
Here, I have split the data up into multiple layers.  As each layer all consist of points, I have created a simple function in order to reuse the same Layer properties.

In [ ]:
def layer(df,color):
    
    dfpd = df.sample(0.5).limit(1000).to_pandas()
    return pdk.Layer(
            'ScatterplotLayer',
            data=dfpd,
            get_position='[LON, LAT]',
            get_color=color,
            get_radius=10,
            radiusScale=100,
            pickable=True)

You will generate a list of regions which will be now our new filter.  the previous **TYPE** filter will not be used, as we will overlay each type as a different layer.

In [ ]:
reg = snames.select('REGION').distinct()
reg

Below is the streamlit code for the new mullt layered pydeck point map.  Please note that there is one tool tip which works for **ALL LAYERS**.  In terms of best practices, keep each layer consistant in terms of naming conventions if you want them to be pickable for tool tip purposes.

In [ ]:
import pydeck as pdk

## create a select box for each distinct region
reg = snames.select('REGION').distinct()
selected_region = st.selectbox('Select Regions',reg)

## filter the dataframe by region
snamesf = snames.filter(col('REGION')==selected_region)

### use filtered dataframe to create 6 datafreames - each filtered to a different type
landform = snamesf.filter(col('TYPE')=='landform')
transportNetwork = snamesf.filter(col('TYPE')=='transportNetwork')
landcover = snamesf.filter(col('TYPE')=='landcover')
hydrography = snamesf.filter(col('TYPE')=='hydrography')
other = snamesf.filter(col('TYPE')=='other')
populatedPlace = snamesf.filter(col('TYPE')=='populatedPlace')

### create the center point
center = snamesf.agg(avg('LAT'),avg('LON'))
LAT = center.collect()[0][0]
LON = center.collect()[0][1]
st.write(center)


#### render the map for each layer by calling the layer function that was previously created
landcover_l = layer(landcover,'[41,181,232]')
transport_l = layer(transportNetwork,'[17,86,127]')
landform_l = layer(landform,'[0,0,0]')
other_l = layer(other,'[138,153,158]')
hydrography_l = layer(hydrography,'[113,221,220]')

#### visualise the  results in pydeck.
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=7,
        height=400
        ),
    
layers= [landcover_l,transport_l,landform_l,other_l,hydrography_l], tooltip = {'text':"Place Name: {NAME1}, Type: {TYPE}"}

))

## 2b. Handling Polygons
Let's now utilise polygons.  Polygons consist of multiple points.  For a polygon to be valid, the last point is always the same as the first point.  We will use polygons to filter another dataset which is the urban extents dataset.  This dataset has 2 location fields - 

- **GEOMETRY**.  This column can be aligned to any grid reference system - in this case, its aligned to the British National Reference system.  You will also note that the data is stored in 'WKT'(Well Known Text) Format.  
- **GEOGRAPHY** - Same as before, The Geography column models the earth as though it were a perfect sphere and follows the WGS 84 standard.  The data is stored as geojson format. 

In [ ]:
urban_extents = session.table('URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL')
urban_extents.limit(5)

You will note that the geography datatype consists of polygons containing both **multi polygons** and **polygons**.  Muilti Polygons, nest several polygons as one object.   the Pydeck Library can only visualise polygons which are not nested as multipolygons.  To resolve this,  let's create simple function to transform all multi polygons to polygons so it's easier for **pydeck** to handle.  This involves **flattening** the multi polygons into multiple rows.  Snowflake's semi structure support allows this to happen.

In [ ]:
def polygon(data):
    # create a new data frame filter the dataframe where the type in each geography field contains the word 'Polygon'
    dataP = data.filter(call_function('ST_ASGEOJSON',col('GEOGRAPHY'))['type'].astype(StringType())=='Polygon')
    # create a new dataframe and Filter the dataframe where the type in each geography field contains the word 'Multi Polygon'
    dataM = data.filter(call_function('ST_ASGEOJSON',col('GEOGRAPHY'))['type'].astype(StringType())=='MultiPolygon')

    ## use the join table function to flatten the multi polygon into one row per polygon
    dataM = dataM.join_table_function('flatten',
                                        call_function('ST_ASGEOJSON',
                                        col('GEOGRAPHY'))['coordinates']).drop('SEQ',
                                                                               'KEY',
                                                                               'PATH',
                                                                               'INDEX',
                                                                               'THIS')                                                                                                        
    
    ## With the flattend results, create a new valid geography object with the type 'Polygon'
    dataM = dataM.with_column('GEOGRAPHY',
                                to_geography(object_construct(lit('coordinates'),
                                                        to_array('VALUE'),
                                                        lit('type'),
                                                        lit('Polygon')))).drop('VALUE')

    ### return both the converted polygons (dataM) as well as the already single polygons (dataP) into one dataframe

    return dataM.union(dataP).with_column_renamed('GEOGRAPHY','POLYGON')

Now we have our new function to convert the polygons, we can render them easily in pydeck.

- Run the cell below which uses the new function. 
- Double click in the geography column to see the result.  

As we have one polygon per row, we might want to consider the handling of the metrics.  Below, we have added an **AREA_KM** column which will return area per polygon rather than the Area_hectares which returned the sum of all polygons inside the multi polygon.

Finally we will wwrite a **new table** using the **write** command and will call it **Urban Extents**. 

In [ ]:
raw_data = session.table('URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL')
dataset = polygon(raw_data.drop('GEOMETRY'))
dataset = dataset.with_column('AREA_KM',div0(call_function('ST_AREA',col('POLYGON')),1000).astype(DecimalType(8,2)))
dataset = dataset.with_column('AREA_HECTARES',div0(call_function('ST_AREA',col('POLYGON')),10000).astype(DecimalType(8,2)))
dataset.write.mode('overwrite').save_as_table("URBAN_EXTENTS")
dataset = session.table('URBAN_EXTENTS')

Before we see the results, run the code below to create a new tooltip which has HTML styling.

In [ ]:
tooltip = {
   "html": """<b>Name:</b> {NAME1_TEXT} <br> <b>Area KM:</b> {AREA_KM} <br> <b>Area Hectares:</b> {AREA_HECTARES}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

Below are the results of the polygons.

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()
from snowflake.snowpark.functions import *
st.subheader('OS Urban Extents - Built up Extents for cities, towns and villages')
### create a filter dropdown using distinct urban extent values

filter = dataset.select('NAME1_TEXT').distinct()
filter = st.selectbox('Choose Town:',filter)
data = dataset.filter(col('NAME1_TEXT')==filter)

### create a center point - this time using the centroid method as we are visualising one polygon at a time
centre = data.with_column('CENTROID',call_function('ST_CENTROID',col('POLYGON')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]


# convert the dataframe to pandas and use a pandas lamda function to extract the coordinates out of each polygon.  
##pydeck only requires sets of coordinates in arrays, not the polygon itself

datapd = data.to_pandas()

datapd["coordinates"] = datapd["POLYGON"].apply(lambda row: json.loads(row)["coordinates"])



# Create data layer for each polygon
data_layer = pdk.Layer(
    "PolygonLayer",
    datapd,
    opacity=0.3,
    get_polygon="coordinates", 
    filled=True,
    get_fill_color=[255, 0, 127],
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=13,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[data_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    
st.pydeck_chart(r, use_container_width=True)



## 2c. Handling Linestrings
Line strings consist of multiple points to form a line. Similar to polygons, you can also have both multi line strings and line strings.
linestring handling is of similar nature to polygons.  Pydeck does not supported 'nested' line string - each line needs to be rendered in a seperate row.  Therefore we will create another function - similar as before to convert multi linestrings to linestrings.

In [ ]:
def linestring(data):
    # create a new data frame filter the dataframe where the type in each geography field contains the word 'Line String'
    dataP = data.filter(call_function('ST_ASGEOJSON',col('GEOGRAPHY'))['type'].astype(StringType())=='LineString')
    # create a new dataframe and Filter the dataframe where the type in each geography field contains the word 'Multi Line String'
    dataM = data.filter(call_function('ST_ASGEOJSON',col('GEOGRAPHY'))['type'].astype(StringType())=='MultiLinesting')
    ## use the join table function to flatten the multi Line String into one row per Line String
    dataM = dataM.join_table_function('flatten',
                                        call_function('ST_ASGEOJSON',
                                        col('GEOGRAPHY'))['coordinates']).drop('SEQ',
                                                                               'KEY',
                                                                               'PATH',
                                                                               'INDEX',
                                                                               'THIS')
    
    ## With the flattend results, create a new valid geography object with the type 'Line String'.                                                                                                        
    dataM = dataM.with_column('GEOGRAPHY',
                                to_geography(object_construct(lit('coordinates'),
                                                        to_array('VALUE'),
                                                        lit('type'),
                                                        lit('LineString')))).drop('VALUE')
    ### return both the converted linestrings (dataM) as well as the already single linestrings (dataP) into one dataframe
    return dataM.union(dataP).with_column_renamed('GEOGRAPHY','LINESTRING')

A perfect dataset to visualise linestrings is the vehicle network.  Below we will put the multi linestrings through the line string function and then persist the results into a new table called **ROAD_LINKS**

In [ ]:
road_link = session.table('ROAD_NETWORK__GREAT_BRITAIN_OPEN_ROADS.PRS_OPENROADS_SCH.PRS_ROAD_LINK_TBL')
road_link.limit(5)

dataset2 = linestring(road_link)
dataset2.write.mode('overwrite').save_as_table("ROAD_LINKS")
dataset2.limit(5)

From a **Pydeck** prospective, to visualise line strings, you will use the **PathLayer**.  Remember this time we are visualising lines, so the line width may more significant here

In [ ]:
#Path layer for road network
network_layer  = pdk.Layer(
        type="PathLayer",
        data=datapd,
        pickable=True,
        get_color=[170, 74, 68],
        width_scale=5,
        opacity = 1,
        width_min_pixels=2,
        get_path="coordinates",
        get_width=2,
)

The filter used for this data frame is based on road classifaction number and road type.  You will be able to select both of these and it will draw the road.  This illustration will only draw one row at a time.  The filtering works the same way as before.

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()
from snowflake.snowpark.functions import *

dataset2 = session.table('ROAD_LINKS')
dataset2 = dataset2.filter(col('ROAD_CLASSIFICATION_NUMBER')!='None')

filter = dataset2.select('ROAD_CLASSIFICATION').distinct()
filter = st.selectbox('Choose Road Type:',filter)

tooltip = {
   "html": """<b>Name:</b> {NAME_1} <br> <b>Form of Way:</b> {FORM_OF_WAY} <br> <b>Length:</b> {LENGTH}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

#Path layer for road network
network_layer  = pdk.Layer(
        type="PathLayer",
        data=datapd,
        pickable=True,
        get_color=[170, 74, 68],
        width_scale=5,
        opacity = 1,
        width_min_pixels=2,
        get_path="coordinates",
        get_width=2,
)

data = dataset2.filter(col('ROAD_CLASSIFICATION')==filter)
filter2 = data.select('ROAD_CLASSIFICATION_NUMBER').distinct()
filter2 = st.selectbox('Choose Classification Number:',filter2)
data = data.filter(col('ROAD_CLASSIFICATION_NUMBER')==filter2)
centre = data.with_column('CENTROID',call_function('ST_CENTROID',col('LINESTRING')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]
# Populate dataframe from query

datapd = data.to_pandas()

datapd["coordinates"] = datapd["LINESTRING"].apply(lambda row: json.loads(row)["coordinates"])

st.write('OS Open Roads')
st.write(datapd)



# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=13,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[network_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    

st.pydeck_chart(r)


# 3. Create a Spatial Filter
Previously, we leveraged the urban extents visualise polygons.  This time, we will use the same polygons to filter line strings.  This is performed by using a spatial filter.  We will also  view multiple layers again - this time however, containing points and lines.  The points represent the road nodes and the lines represent the road links.

Filtering is performed by creating an inner join between the towns and the roads.  **ST_INTERSECTS** joins only when a road crosses a town.  The filter this time are the towns.  Run the code below to view only road links that cross over the town

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()
from snowflake.snowpark.functions import *

filter = dataset.select('NAME1_TEXT').distinct()
filter = st.selectbox('Choose Town:',filter,4)
data = dataset.filter(col('NAME1_TEXT')==filter)
dataset3 = session.table('ROAD_LINKS')

#### the join to only view results if the road crosses over the town

datajoined = data.join(dataset3,call_function('ST_INTERSECTS',data['POLYGON'],dataset3['LINESTRING']))


centre = datajoined.with_column('CENTROID',call_function('ST_CENTROID',col('POLYGON')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

datajoined = datajoined.drop('POLYGON')

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]
# Populate dataframe from query

datapd = datajoined.to_pandas()

datapd["coordinates"] = datapd["LINESTRING"].apply(lambda row: json.loads(row)["coordinates"])



# Create data layer - this where the geometry is likely failing - column is now called geometry to match geopandas default
network_layer  = pdk.Layer(
        type="PathLayer",
        data=datapd,
        pickable=True,
        get_color=[170, 74, 68],
        width_scale=5,
        opacity = 1,
        width_min_pixels=2,
        get_path="coordinates",
        get_width=2,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=13,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[network_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    
st.pydeck_chart(r, use_container_width=True)



Let's now combine with the Road Nodes - these are points.  We can use the same intersect join, to map our nodes with the road details.  The dataframe has been modified slightly so it is consistant with the previous dataframe.  Tooltips for multi layers require the same naming parameters, so padding out additional columns with 'N/A' is helpful

In [ ]:
road_nodes = session.table('ROAD_NETWORK__GREAT_BRITAIN_OPEN_ROADS.PRS_OPENROADS_SCH.PRS_ROAD_NODE_TBL')
road_nodes = road_nodes.withColumnRenamed('FORM_OF_ROAD_NODE','FORM_OF_WAY')
road_nodes = road_nodes.withColumn('NAME_1',lit('N/A'))
road_nodes = road_nodes.withColumn('LENGTH',lit('N/A'))
road_nodes.with_column_renamed('GEOGRAPHY','POINT').write.mode('overwrite').save_as_table("ROAD_NODES")


In [ ]:
road_nodes = session.table('road_nodes')
nodes_filtered = road_nodes.join(datajoined.select('LINESTRING'),call_function('ST_INTERSECTS',
                                         datajoined['LINESTRING'],road_nodes['POINT'])).drop('LINESTRING')


st.write(nodes_filtered.limit(10))

Below you should now see a multi layer map consisting of the road nodes as well as the road segments themselves

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import *
from snowflake.snowpark.context import get_active_session
session = get_active_session()



tooltip = {
   "html": """<b>Name:</b> {NAME_1} <br> <b>Form of Way:</b> {FORM_OF_WAY} <br> <b>Length:</b> {LENGTH}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}




road_links = session.table('ROAD_LINKS')
urban_extents = session.table('URBAN_EXTENTS')
road_nodes = session.table('ROAD_NODES')




filter = urban_extents.select('NAME1_TEXT').distinct()
filter = st.selectbox('Choose Town:',filter,7)
data = urban_extents.filter(col('NAME1_TEXT')==filter)


datajoined = data.join(road_links,call_function('ST_INTERSECTS',data['POLYGON'],road_links['LINESTRING']))



nodes_filtered = road_nodes.join(datajoined.select('LINESTRING'),call_function('ST_INTERSECTS',
                                         datajoined['LINESTRING'],road_nodes['POINT'])).drop('LINESTRING')




centre = datajoined.with_column('CENTROID',call_function('ST_CENTROID',col('POLYGON')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

datajoined = datajoined.drop('POLYGON')

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]
# Populate dataframe from query

datapd = datajoined.to_pandas()

datapd["coordinates"] = datapd["LINESTRING"].apply(lambda row: json.loads(row)["coordinates"])



# Create data layer - this where the geometry is likely failing - column is now called geometry to match geopandas default
network_layer  = pdk.Layer(
        type="PathLayer",
        data=datapd,
        pickable=True,
        get_color=[170, 74, 68],
        width_scale=5,
        opacity = 1,
        width_min_pixels=2,
        get_path="coordinates",
        get_width=2,
)

datajoined = datajoined.drop('POLYGON')

nodes_filtered_2 = nodes_filtered.with_column('LAT',call_function('ST_Y',col('POINT')))
nodes_filtered_2 = nodes_filtered_2.with_column('LON',call_function('ST_X',col('POINT')))
nodes_filtered_2 = nodes_filtered_2.drop('POINT')
# Create data layer - this where the geometry is likely failing - column is now called geometry to match geopandas default
nodes = pdk.Layer(
            'ScatterplotLayer',
            data=nodes_filtered_2.to_pandas(),
            get_position='[LON, LAT]',
            get_color='[41,181,232]',
            get_radius=20,
            pickable=True)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=13,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[network_layer,nodes],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    
st.pydeck_chart(r, use_container_width=True)



To avoid reapplying the same linestring transformations let's persist this information.  We will then speed up the search capability of the table on geo by applying **Search Optimisation**

In [ ]:
ALTER TABLE ROAD_LINKS ADD SEARCH OPTIMIZATION ON GEO(LINESTRING);

ALTER TABLE URBAN_EXTENTS ADD SEARCH OPTIMIZATION ON GEO(POLYGON);

ALTER TABLE ROAD_NODES ADD SEARCH OPTIMIZATION ON GEO(POINT);

# 4. H3
Snowflake supports the popular H3 indexing system.  It is very easy to convert points to H3 as well as covering polygons.  The simple excercise we will cover today, is converting the points that we used in section 2 of the lab to H3.  We will use the function **H3_POINT_TO_CELL_STRING(point,resolution)**.  

H3 allows for fast processing using multiple reslutions.  H3 works well using gradients.  To do this you might want to influence the RGB colour by creating a 'proportion between 0 and 255'.

This time, we are using SQL to convert the data to H3.  we will use the results of this in our pydeck map

In [ ]:
SELECT H3_POINT_TO_CELL_STRING(GEOGRAPHY,7) H3, TYPE, COUNT(*) COUNT,

(COUNT - MIN(COUNT) OVER (PARTITION BY TYPE)) / 
    NULLIF(MAX(COUNT) OVER (PARTITION BY TYPE) - MIN(COUNT) OVER (PARTITION BY TYPE) , 0) RATIO,
    RATIO * 41 AS R,
    RATIO  * 181 AS G,
    RATIO * 232 AS B



FROM (
select * from POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL) GROUP BY ALL

Here is the new H3 layer which the H3 dataframe will be passed through.  Before we pass the dataframe, we will put it through a **TYPE** filter which the user will select when they run the visualisation

In [ ]:
s_types = h3_lev_5.to_df().select('TYPE').distinct()

filter_type = st.radio('Select Type: ', s_types)


h3pd = h3_lev_5.to_df().filter(col('TYPE')==filter_type).to_pandas()
h3 = pdk.Layer(
        "H3HexagonLayer",
        h3pd,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon="H3",
        get_fill_color=["R","G","B"],
        line_width_min_pixels=0,
        opacity=0.4)

This is the rendered H3 output

**Exercise** - Experiment with the different resolutions.  The higher the resolution, the smaller the hexagon.  The lower the resolution, the larger the hexagon.


In [ ]:
s_types = h3_lev_5.to_df().select('TYPE').distinct()

filter_type = st.radio('Select Type: ', s_types,2)


h3pd = h3_lev_5.to_df().filter(col('TYPE')==filter_type).to_pandas()
h3 = pdk.Layer(
        "H3HexagonLayer",
        h3pd,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon="H3",
        get_fill_color=["R","G","B"],
        line_width_min_pixels=0,
        opacity=0.4)

tooltip = {
   "html": """<b>H3:</b> {H3} <br> <b>Count:</b> {COUNT}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=600
        ),
    
layers= [h3], tooltip = tooltip

))


### H3 Resolution changes

You can dynamically lower the resolution easily see below an example of this using **H3_CELL_TO_PARENT_STRING**

Remember, if you lower the resolution, you will need to re group the data - and consider how you would like to aggregate the measures.

In [ ]:
s_types = h3_lev_5.to_df().select('TYPE').distinct()

filter_type2 = st.selectbox('Select Type: ', s_types)
res = st.slider('Change Resolution:',1,7,7)

h3 = h3_lev_5.to_df().with_column('H3',call_function('H3_CELL_TO_PARENT',col('H3'),res))

h3pd = h3.filter(col('TYPE')==filter_type2).group_by('H3').agg(max('R').alias('R'),
                                                          max('G').alias('G'),
                                                          max('B').alias('B'),
                                                          count('COUNT').alias('COUNT')).to_pandas()
h3 = pdk.Layer(
        "H3HexagonLayer",
        h3pd,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon="H3",
        get_fill_color=["R","G","B"],
        line_width_min_pixels=0,
        opacity=0.4)

tooltip = {
   "html": """<b>H3:</b> {H3} <br> <b>Count:</b> {COUNT}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=600
        ),
    
layers= [h3], tooltip = tooltip

))


# 5. Search and filter on GEOHASH 

Here, we are extracting the UK data from a worldwide dataset of over **2.6B records**.  We will filter by geohash.  You can view https://www.movable-type.co.uk/scripts/geohash.html to search for valid geohashes.  As this is a large dataset, we will use a larger warehouse.  We will then join the data to the built up extents (which we used previously) - this will effectively 'geocode' the buildings to a town which can then be used later in our filtering.  

This may take around 5 minutes to complete

In [ ]:
USE WAREHOUSE X_LARGE_HEAVY_LIFT;
CREATE OR REPLACE TABLE BUILDINGS AS 

SELECT A.*,B.NAME1_TEXT FROM

(SELECT *,ST_GEOHASH(GEOMETRY,2) GEOHASH FROM OVERTURE_MAPS__BUILDINGS.CARTO.BUILDING WHERE GEOHASH IN('gf','gb''gc','u1')) A

INNER JOIN

URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_EXTENTS_TBL B

ON

ST_INTERSECTS(A.GEOMETRY,B.GEOGRAPHY) 

ORDER BY NAME1_TEXT




;




ALTER WAREHOUSE X_LARGE_HEAVY_LIFT SUSPEND;

USE WAREHOUSE DEFAULT_WH;



In [ ]:
ALTER TABLE BUILDINGS ADD SEARCH OPTIMIZATION ON GEO(GEOMETRY);

Finally we will render ther results of this in streamlit - the same as before, I have chosen the town extents to filter the building polygons - so this time one big polygon filters lots of smaller polygons.

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()
from snowflake.snowpark.functions import *

tooltip = {
   "html": """<b>CLASS:</b> {CLASS} <br> <b>SUBTYPE:</b> {SUBTYPE} <br> <b>Area Hectares:</b> {AREA_HECTARES}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}
BUILDINGS = session.table('BUILDINGS')
filter = BUILDINGS.select('NAME1_TEXT').distinct()
filter = st.selectbox('Choose Town:',filter, 6)

BUILDINGS = BUILDINGS.filter(col('NAME1_TEXT')==filter).select('GEOMETRY','CLASS','SUBTYPE')

centre = session.table('URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_EXTENTS_TBL')
centre = centre.filter(col('NAME1_TEXT')==filter)
centre = centre.with_column('CENTROID',call_function('ST_CENTROID',col('GEOGRAPHY')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))




centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]
# Populate dataframe from query

datapd = BUILDINGS.to_pandas()

datapd["coordinates"] = datapd["GEOMETRY"].apply(lambda row: json.loads(row)["coordinates"])

st.write('Buildings in a town')

# Create data layer - this where the geometry is likely failing - column is now called geometry to match geopandas default
data_layer = pdk.Layer(
    "PolygonLayer",
    datapd,
    opacity=0.3,
    get_polygon="coordinates", 
    filled=True,
    get_fill_color=[255, 0, 127],
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=13,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[data_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    
st.pydeck_chart(r, use_container_width=True)



# Conclusion

You have now completed a hands on lab which explores how you can use snowflake's geospatial capabilities to visualise location using streamlit inside a Snowflake Powered Notebook.

### What we have Covered examples of the following:

- Data Formats, Geo Data Types
- Points, Linestrings, Polygons
- H3
- Visualising data with Streamlit and Pydeck
- Spatial Joins and calculations
- Search optimisation
- Geohash

# Streamlit Example

- Navigate back to the home page and open **Projects>>Streamlit**.

- Open the ROAD_NETWORK streamlit example which combines points, linestrings and polygons together to illustrate blueprints of towns.  This will also give you a starting point to create many streamlit examples of your own.
